In [47]:
import html
from bs4 import BeautifulSoup
from glob import glob
from itertools import chain
from pathlib import Path
import pandas as pd
import requests
import numpy as np
from tqdm import tqdm
import json

## Arabic - Hebrew

In [16]:
arabit_browse_template = "https://arabit.me/page/{page_number}/"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}
page_htmls = []
for page_number in tqdm(np.arange(1, 232)):
    page_url = arabit_browse_template.format(page_number=page_number)
    page_html = requests.get(page_url, headers=headers)
    page_htmls.append(page_html)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 231/231 [05:58<00:00,  1.55s/it]


In [17]:
for i, page in enumerate(page_htmls):
    with open(f'data/arabit/browse/{str(i+1)}', 'w') as f:
        f.write(page.text)

In [23]:
word_pages = []
for page in page_htmls:
    soup = BeautifulSoup(page.text, 'html.parser')

    entries = soup.find_all('h2', class_='entry-title')
    for entry in entries:
        link = entry.find('a').get("href")
        title = entry.text.strip()
        word_pages.append((title, link))

In [24]:
wp_df = pd.DataFrame(word_pages, columns=['title', 'link'])
wp_df.tail()

,title,link
2297,איכס,https://arabit.me/%d7%90%d7%99%d7%9b%d7%a1/
2298,כלוא,https://arabit.me/%d7%9b%d7%9c%d7%95%d7%90/
2299,נעל ישנה,https://arabit.me/%d7%a0%d7%a2%d7%9c-%d7%99%d7...
2300,טבח,https://arabit.me/%d7%98%d7%91%d7%97/
2301,נביא,https://arabit.me/%d7%a0%d7%91%d7%99%d7%90/


In [25]:
wp_df.shape

(2302, 2)

In [28]:
wp_df.title.nunique()

2167

In [27]:
wp_df.link.nunique()

2302

In [26]:
wp_df.to_csv('data/arabit/pages.csv', index=False)

In [33]:
import os

In [35]:
from concurrent.futures import ThreadPoolExecutor

def save_page(params):
    i, title, link = params
    output_path = f'data/arabit/word_pages/{str(i+1)}.html'
    if not os.path.exists(output_path): 
        word_page_html = requests.get(link, headers=headers)
        with open(output_path, 'w') as f:
            f.write(word_page_html.text)

with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(save_page, wp_df.itertuples()), total=len(wp_df)))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2302/2302 [17:21<00:00,  2.21it/s]


In [95]:
import re
import unicodedata


def normalize_text(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')


def find_image(soup):
    yoast = json.loads(soup.find("script", class_="yoast-schema-graph").string)
    for elem in yoast["@graph"]:
        if elem.get("@type") == 'ImageObject':
            return elem["caption"], elem["url"]
        
        
SEPS = "();,"
rsplit = re.compile("\\"+"|\\".join(SEPS)).split


def find_arabic_words(s):
    pattern = r'[\u0600-\u06FF\s]+'
    return [ w for w in re.findall(pattern, s) if len(w)>1 ]


words = []
for i, title, link in tqdm(list(wp_df.itertuples()), total=len(wp_df)):
    #print(i, title, link)
    wp_path = f'data/arabit/word_pages/{str(i+1)}.html'
    with open(wp_path, 'r') as f:
        wp_html = f.read()
    soup = BeautifulSoup(wp_html, 'html.parser')
    try:
        caption, image = find_image(soup)
    except:
        print(f"Failed: {link}")
        continue
    caption_part = normalize_text(rsplit(caption)[0].strip(" “\""))
    #print(caption_part, image)
    
    #find in body
    body = soup.find("body")
    arabic_word = None
    for line in body.text.split("\n"):
        line_norm = normalize_text(line)
        if caption_part in line_norm:
            arabic_words = find_arabic_words(line)
            if arabic_words:
                arabic_word = arabic_words[0]
                break
    #print(arabic_word)
    words.append((i+1, title, link, image, caption, caption_part, arabic_word))


 26%|████████████████████████████████████████▌                                                                                                                | 610/2302 [00:27<01:01, 27.45it/s]

Failed: https://arabit.me/%d7%9e%d7%9b%d7%95%d7%9c%d7%aa/


 29%|███████████████████████████████████████████▋                                                                                                             | 657/2302 [00:30<01:05, 25.02it/s]

Failed: https://arabit.me/%d7%a4%d7%9c%d7%99%d7%a9%d7%94/


 68%|██████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                 | 1558/2302 [01:10<00:26, 28.11it/s]

...

 70%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                             | 1608/2302 [01:12<00:31, 21.88it/s]

Failed: https://arabit.me/%d7%9e%d7%a6%d7%95%d7%99%d7%9f/
Failed: https://arabit.me/%d7%90%d7%97%d7%95%d7%a9%d7%9c%d7%95%d7%a7%d7%99/
Failed: https://arabit.me/%d7%a9%d7%99%d7%a4%d7%95%d7%a8/


 77%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                   | 1768/2302 [01:19<00:24, 21.63it/s]

Failed: https://arabit.me/%d7%a4%d7%99%d7%aa%d7%aa-%d7%9c%d7%a4%d7%94-2/


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2302/2302 [01:44<00:00, 22.12it/s]


## Download images


In [102]:
image_urls = [iu for w in words if (iu:=w[3]).startswith("http")]

In [104]:
from concurrent.futures import ThreadPoolExecutor
import shutil

def save_image(image_url):
    image_name = image_url.split("/")[-1]
    output_path = f'data/arabit/images/{image_name}'
    if not os.path.exists(output_path): 
        r = requests.get(image_url, headers=headers, stream=True)
        if r.status_code == 200:
            with open(output_path, 'wb') as f:
                for chunk in r:
                    f.write(chunk)

with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(save_image, image_urls), total=len(image_urls)))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2286/2286 [02:36<00:00, 14.61it/s]


## Reduce image size

In [117]:
from PIL import Image
OUTPUT_FOLDER = 'data/arabit/images_smaller/'
def compress_image(image, max_size=100000, scale=0.9):
    output_path = os.path.join(OUTPUT_FOLDER, image)
    if os.path.exists(output_path):
        return
    image_path = os.path.join('data/arabit/images/', image)
    img = Image.open(image_path)

    if os.path.getsize(image_path) <= max_size:
        shutil.copy(image_path, output_path)
        return
    # If image size is more than max size provided, reduce the quality of image 
    while os.path.getsize(image_path) > max_size:
        width, height = img.size
        img.thumbnail((width*scale, height*scale))
        image_path = os.path.join(OUTPUT_FOLDER, image)
        img.save(image_path,optimize=True, quality=85)

for image in tqdm(os.listdir('data/arabit/images/')):
    filename, extension = os.path.splitext(image)
    compress_image(image)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2254/2254 [06:35<00:00,  5.69it/s]


In [119]:
from PIL import Image
OUTPUT_FOLDER = 'data/arabit/images_xs/'
def compress_image(image, max_size=25000, scale=0.9):
    output_path = os.path.join(OUTPUT_FOLDER, image)
    if os.path.exists(output_path):
        return
    image_path = os.path.join('data/arabit/images_smaller/', image)
    img = Image.open(image_path)

    if os.path.getsize(image_path) <= max_size:
        shutil.copy(image_path, output_path)
        return
    # If image size is more than max size provided, reduce the quality of image 
    while os.path.getsize(image_path) > max_size:
        width, height = img.size
        img.thumbnail((width*scale, height*scale))
        image_path = os.path.join(OUTPUT_FOLDER, image)
        img.save(image_path,optimize=True, quality=75)

for image in tqdm(os.listdir('data/arabit/images_smaller/')):
    if '.DS_Store' in image or '.ipynb' in image:
        continue
    filename, extension = os.path.splitext(image)
    compress_image(image)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2255/2255 [02:54<00:00, 12.94it/s]


## Find matching audio for each page

In [204]:
url_re = re.compile("https?:\\/\\/(?:w+\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)")
soundcloud_re = url_re = re.compile(r"https?://(?:[\w\.]+\.)?soundcloud\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&/=]*)")

In [206]:
sounds = []
for i, title, link in tqdm(list(wp_df.itertuples()), total=len(wp_df)):
    #print(i, title, link)
    wp_path = f'data/arabit/word_pages/{str(i+1)}.html'
    with open(wp_path, 'r') as f:
        wp_html = f.read()
    soup = BeautifulSoup(wp_html, 'html.parser')
    article = soup.find("article")
    sound_link = re.findall(soundcloud_re, article.prettify())
    sound = article.find("iframe", title=re.compile(".* by Arabit"))
    if sound is not None:
        sound = sound.get("title")
    else:
        sound = None
    sounds.append((i+1, title, sound, sound_link))

100%|███████████████████████████████████████████████████████████████████████████████| 2302/2302 [01:51<00:00, 20.63it/s]


In [209]:
len([1 for s in sounds if s[2] is not None])

683

In [210]:
len([1 for s in sounds if s[3]!=[]])

1129

In [156]:
len([1 for s in sounds if s[2] is not None or (s[3] is not None and s[3]!=[])])

2302

In [157]:
len(os.listdir('data/arabit/soundcloud/'))

1143

In [158]:
len(os.listdir('data/arabit/word_pages/'))

2303

In [239]:
r = requests.get('https://on.soundcloud.com/s1Lxr?_imcp=1')

## Scrape title of soundcloud links and match to soundcloud file names

In [247]:
soundcloud_titles = {}
soundcloud_link_refer = {}
for i, title, sound, sound_link in sounds:
    for link in sound_link:
        clean_link = re.sub('\?.*$', '', link)
        if clean_link in soundcloud_titles or '/player/' in link:
            continue
        if '//on.' in clean_link:
            link_to_request = clean_link + '?_impcp=1'
        else:
            link_to_request = clean_link
        print(link)
        print(link_to_request)
        r = requests.get(link_to_request)#, headers=headers, allow_redirects=True)
        r.encoding = r.apparent_encoding
        print(r.url)
        soundcloud_link_refer[link] = r.url
        soup = BeautifulSoup(r.text, 'html.parser')
        title = soup.find("title").text
        print(title)
        soundcloud_titles[clean_link] = (r.url, title)

https://on.soundcloud.com/s1Lxr?fbclid=IwAR2rKTdcdVf0Q2KqfKO5sd8ryS9v8YkyM4vTCNf1mM5GDWK7PFamfmh4-tI
https://on.soundcloud.com/s1Lxr?_impcp=1
https://soundcloud.com/listen2arabit/d5yl754v5w1h?si=9bac91b8a0e2456da7e61b40cdd345bf&utm_source=clipboard&utm_medium=text&utm_campaign=social_sharing
Stream שכרן (תודה) by Arabit | Listen online for free on SoundCloud
https://on.soundcloud.com/Jag6v?fbclid=IwAR2DYrZ5SIN2FY_glT5_Dk9PgcXtkwbg0fjPc6ii808n_QMext7GPH73qjI
https://on.soundcloud.com/Jag6v?_impcp=1
https://soundcloud.com/listen2arabit/n1c2fhnql01s?si=9bac91b8a0e2456da7e61b40cdd345bf&utm_source=clipboard&utm_medium=text&utm_campaign=social_sharing
Stream מע׳ארה (מערה) by Arabit | Listen online for free on SoundCloud
https://on.soundcloud.com/9RqRd?fbclid=IwAR1tQ0S383bBOnmGrl1Ra12LwOxaTUeb79i-3j_nKnPlAvk2vUR77ORw3ko
https://on.soundcloud.com/9RqRd?_impcp=1
https://soundcloud.com/listen2arabit/k0obd6mtnt47?si=9bac91b8a0e2456da7e61b40cdd345bf&utm_source=clipboard&utm_medium=text&utm_campaig

In [248]:
len(soundcloud_titles)

515

In [250]:
sc_titles_df = pd.DataFrame([(clean_link, url, title) for clean_link, (url, title) in soundcloud_titles.items()],
                            columns=['clean_link', 'url', 'title'])
sc_titles_df.head()

,clean_link,url,title
0,https://on.soundcloud.com/s1Lxr,https://soundcloud.com/listen2arabit/d5yl754v5...,Stream שכרן (תודה) by Arabit | Listen online f...
1,https://on.soundcloud.com/Jag6v,https://soundcloud.com/listen2arabit/n1c2fhnql...,Stream מע׳ארה (מערה) by Arabit | Listen online...
2,https://on.soundcloud.com/9RqRd,https://soundcloud.com/listen2arabit/k0obd6mtn...,Stream כרסי רג׳אג׳ (כסא נדנדה) by Arabit | Lis...
3,https://on.soundcloud.com/HAA6e,https://soundcloud.com/listen2arabit/3rbrejejj...,Stream אנצ׳ם (הצטרף) by Arabit | Listen online...
4,https://on.soundcloud.com/SDpcs,https://soundcloud.com/listen2arabit/brxcpvenr...,Stream האתף (טלפון) by Arabit | Listen online ...


In [277]:
sc_titles_df.title.iat[0]

'Stream שכרן (תודה) by Arabit | Listen online for free on SoundCloud'

In [251]:
sc_ref_df = pd.DataFrame(soundcloud_link_refer.items(), columns=['link', 'refer_url'])
sc_ref_df.head()

,link,refer_url
0,https://on.soundcloud.com/s1Lxr?fbclid=IwAR2rK...,https://soundcloud.com/listen2arabit/d5yl754v5...
1,https://on.soundcloud.com/Jag6v?fbclid=IwAR2DY...,https://soundcloud.com/listen2arabit/n1c2fhnql...
2,https://on.soundcloud.com/9RqRd?fbclid=IwAR1tQ...,https://soundcloud.com/listen2arabit/k0obd6mtn...
3,https://on.soundcloud.com/HAA6e?fbclid=IwAR1e3...,https://soundcloud.com/listen2arabit/3rbrejejj...
4,https://on.soundcloud.com/SDpcs,https://soundcloud.com/listen2arabit/brxcpvenr...


In [253]:
sc_ref_df.to_csv('data/arabit/soundcloud_link_referrals.csv', index=False)

In [314]:
sc_titles_df.loc[sc_titles_df.clean_link=='https://on.soundcloud.com/bHxPz', 'url'] = 'https://soundcloud.com/listen2arabit/4wp9khxj4tru?si=b8542567dc5341caa02d61cbf46f9967&utm_source=clipboard&utm_medium=text&utm_campaign=social_sharing'
sc_titles_df.loc[sc_titles_df.clean_link=='https://on.soundcloud.com/bHxPz', 'title'] = 'Stream מח\'דראת by Arabit | Listen online for free on SoundCloud'

In [316]:
sc_fname_re = 'Stream (.*) by Arabit | Listen online for free on SoundCloud'
sc_titles_df["title_extracted"] = sc_titles_df.title.str.extract(sc_fname_re).iloc[:, 0].str.strip()
sc_titles_df["local_path"] = 'data/arabit/soundcloud/' + sc_titles_df.title_extracted.str.replace("/", "") + '.mp3'
sc_titles_df['original_link'] = sc_titles_df.url.map(sc_ref_df.set_index("refer_url")["link"])
sc_titles_df.head()

,clean_link,url,title,title_extracted,local_path,original_link
0,https://on.soundcloud.com/s1Lxr,https://soundcloud.com/listen2arabit/d5yl754v5...,Stream שכרן (תודה) by Arabit | Listen online f...,שכרן (תודה),data/arabit/soundcloud/שכרן (תודה).mp3,https://on.soundcloud.com/s1Lxr?fbclid=IwAR2rK...
1,https://on.soundcloud.com/Jag6v,https://soundcloud.com/listen2arabit/n1c2fhnql...,Stream מע׳ארה (מערה) by Arabit | Listen online...,מע׳ארה (מערה),data/arabit/soundcloud/מע׳ארה (מערה).mp3,https://on.soundcloud.com/Jag6v?fbclid=IwAR2DY...
2,https://on.soundcloud.com/9RqRd,https://soundcloud.com/listen2arabit/k0obd6mtn...,Stream כרסי רג׳אג׳ (כסא נדנדה) by Arabit | Lis...,כרסי רג׳אג׳ (כסא נדנדה),data/arabit/soundcloud/כרסי רג׳אג׳ (כסא נדנדה)...,https://on.soundcloud.com/9RqRd?fbclid=IwAR1tQ...
3,https://on.soundcloud.com/HAA6e,https://soundcloud.com/listen2arabit/3rbrejejj...,Stream אנצ׳ם (הצטרף) by Arabit | Listen online...,אנצ׳ם (הצטרף),data/arabit/soundcloud/אנצ׳ם (הצטרף).mp3,https://on.soundcloud.com/HAA6e?fbclid=IwAR1e3...
4,https://on.soundcloud.com/SDpcs,https://soundcloud.com/listen2arabit/brxcpvenr...,Stream האתף (טלפון) by Arabit | Listen online ...,האתף (טלפון),data/arabit/soundcloud/האתף (טלפון).mp3,https://on.soundcloud.com/SDpcs


In [317]:
sc_titles_df[sc_titles_df.local_path.isna()]

,clean_link,url,title,title_extracted,local_path,original_link
221,https://on.soundcloud.com/1XQTk,https://soundcloud.com/listen2arabit/olblsbfid...,SoundCloud - Hear the worldâ€™s sounds,NaN,NaN,https://on.soundcloud.com/1XQTk?fbclid=IwAR3eO...
509,https://soundcloud.com/palestine-nasheed/sfsqd...,https://soundcloud.com/palestine-nasheed/sfsqd...,SoundCloud - Hear the worldâ€™s sounds,NaN,NaN,https://soundcloud.com/palestine-nasheed/sfsqd...


In [318]:
sc_titles_df.to_csv('data/arabit/soundcloud_titles.csv', index=False)

In [319]:
for path in sc_titles_df.local_path:
    if type(path)==str and not os.path.exists(path):
        print(path)

In [351]:
from textdistance import levenshtein
def take_heb(tit):
    if '(' in tit:
        return tit.split('(')[1].strip('()')
    else:
        return tit
page_to_sound_titles = []
for i, page_title, iframe_title, links in sounds:
    titles = list(sc_titles_df[(~sc_titles_df.title_extracted.isna()) & ((sc_titles_df.clean_link.isin(links)) | (sc_titles_df.original_link.isin(links)))].title_extracted.unique())

    if iframe_title:
        clean_title = iframe_title.replace(' by Arabit', '').strip()
        if clean_title not in titles:
            titles.append(clean_title)
    if len(titles)==0:
        continue
    if len(titles)==1:
        chosen_title = titles[0]
    else:
        distances = [levenshtein(page_title, take_heb(st)) for st in titles]
        chosen_title = titles[np.argmin(distances)]
        print(page_title, titles, '-->', chosen_title)
    page_to_sound_titles.append( (i, page_title, chosen_title))    

תגובה ברשת ['תעליק (תגובה ברשת)', 'עליק'] --> תעליק (תגובה ברשת)
תקופת האבן ['אלעצר אלחג׳רי (תקופת האבן)', 'חג׳אר'] --> אלעצר אלחג׳רי (תקופת האבן)
הועיל ['פאד (הועיל)', 'פאאדה'] --> פאד (הועיל)
נעצר (במשטרה) ['אנחבס (נעצר במשטרה)', 'מחבוס'] --> אנחבס (נעצר במשטרה)
תפקיד ['וט׳יפה (תפקיד)', "מוט'פ"] --> וט׳יפה (תפקיד)
פנה בקריאה ['נאשד (פנה בקריאה)', 'נשיד וטני'] --> נאשד (פנה בקריאה)
פרסם ['נשר (פרסם)', 'מנשור'] --> נשר (פרסם)
יום הולדת ['עיד מילאד (יום הולדת)', 'ולאויד', 'ולד'] --> עיד מילאד (יום הולדת)
לימד ['דרס (לימד)', 'מדרסה'] --> דרס (לימד)
מבוהל ['מרעוב (מבוהל)', 'רעב'] --> מרעוב (מבוהל)
שש ['סתה (שש)', 'מסדס'] --> סתה (שש)
גרביים ['כלסאת (גרביים)', 'כלסון'] --> כלסאת (גרביים)
הידרדרות ['אנחטאט (הדרדרות)', 'חט ע ראסה'] --> אנחטאט (הדרדרות)
חסר תקדים ['ע׳יר מסבוק (חסר תקדים)', 'סבאק'] --> ע׳יר מסבוק (חסר תקדים)
התגאה ['אפתח׳ר (התגאה)', "פאח'ר"] --> אפתח׳ר (התגאה)
בגדים ['הדום (בגדים)', 'הדם'] --> הדום (בגדים)
רחבה, כיכר ['באחה (רחבה, כיכר)', 'אסתבאח'] --> באחה (רחבה, כיכר)
מקצוענ

In [355]:
sound_map_fixes_str = '''
תגובה ברשת --> תעליק (תגובה ברשת)
תקופת האבן  --> אלעצר אלחג׳רי (תקופת האבן)
הועיל  --> פאד (הועיל)
נעצר (במשטרה)  --> אנחבס (נעצר במשטרה)
תפקיד  --> וט׳יפה (תפקיד)
פנה בקריאה --> נאשד (פנה בקריאה)
פרסם --> נשר (פרסם)
יום הולדת  --> עיד מילאד (יום הולדת)
לימד  --> דרס (לימד)
מבוהל --> מרעוב (מבוהל)
שש --> סתה (שש)
גרביים --> כלסאת (גרביים)
הידרדרות  --> אנחטאט (הדרדרות)
חסר תקדים  --> ע׳יר מסבוק (חסר תקדים)
התגאה --> אפתח׳ר (התגאה)
בגדים --> הדום (בגדים)
רחבה, כיכר [--> באחה (רחבה, כיכר)
מקצוענות --> אחתראף (מקצוענות)
תרמית --> ח׳דיעה (תרמית)
אני נשבע באל העצום--> ואללה אלעט׳ים (אני נשבע באל העצום)
סוגיה --> מסאלה (סוגיה)
שולחן אוכל --> ספרה (שולחן אוכל)
מציל --> מנקד׳ בחרי (מציל)
אדם מבוגר --> כביר אלסנ (אדם מבוגר)
מאוים --> מהדד (מאוים)
לונה פארק --> מדינה מלאהי (לונה פארק)
עולם המתים והשדים --> אלעאלם אלספלי
גנוב --> מסרוק
שמח --> סריר
קוקוס --> ג׳וז הנדי
התעורר --> פאק
התעכב --> תאח׳ר
טוב --> ג׳יד
שוק (איבר) --> סוק
רצה --> אראד
הרוג --> קתיל
שים מבטחך באל --> תוכל עלא אללה
טובע --> ע'רקאן
סיפורים ובילויים בלילה --> סמר
קדוש --> מקדס
שלווה --> אטמאנאן
שלב --> מרחלה
מבוקש --> מטלוב
יישום --> תטביק
עזר --> עאן
מרוצה --> קריר אלעין
האכיל --> טעמא
הפגנה --> מט'אהרה
תבשיל --> טביח'
תל אביב --> תל אביב
ברואים בצלם האל --> ח'לקת רבנא
שר --> וזיר
מזונות --> דעמ אלטפל
החזיר --> רד
מודאג --> קלקאן
מנה, ארוחה  --> וג'בה
יש לך מזל --> חמאתכ בתחבכ
אדם זר --> אג'נבי
חזון --> ראיא
שחה --> סבח
גרגר חיטה --> חבת קמח
עוינות --> עדאוה
מברג --> מפכ
חזקו אלו את אלו --> שדוא בעצ'כם
מה חסר לך? --> איש נאקצכ
כמה חבל --> יא ח'סארה
מונה --> עדאד
מודעות --> ועי
משרת --> ח'אדם
צר --> צ'יק
החליק --> אנזלק
קדמה --> רקי
רקד --> רקצ
אירוח --> צ'יאפה
ילדים --> ולאויד
העז --> אסתרג'א
עד ראייה --> שאהד עיאן
בריחה --> הרוב
'''

sound_map_fixes = {}
for line in sound_map_fixes_str.split('\n'):
    if line.strip():
        page_title, true_sound_title = line.split('-->')
        sound_map_fixes[page_title.strip()] = true_sound_title.strip()

fixed_ptst = [(i, pt,sound_map_fixes.get(pt, ct)) for i, pt, ct in page_to_sound_titles]
soundcloud_page_mapping = pd.DataFrame(fixed_ptst, columns=["page_num", "page_title", "sound_title"])
soundcloud_page_mapping["local_path"] = 'data/arabit/soundcloud/' + soundcloud_page_mapping.sound_title.str.replace("/", "") + '.mp3'
soundcloud_page_mapping.head()

,page_num,page_title,sound_title,local_path
0,1,תודה,שכרן (תודה),data/arabit/soundcloud/שכרן (תודה).mp3
1,2,מערה,מע׳ארה (מערה),data/arabit/soundcloud/מע׳ארה (מערה).mp3
2,3,כיסא נדנדה,כרסי רג׳אג׳ (כסא נדנדה),data/arabit/soundcloud/כרסי רג׳אג׳ (כסא נדנדה)...
3,4,הצטרף,אנצ׳ם (הצטרף),data/arabit/soundcloud/אנצ׳ם (הצטרף).mp3
4,5,טלפון,האתף (טלפון),data/arabit/soundcloud/האתף (טלפון).mp3


In [356]:
soundcloud_page_mapping.to_csv('data/arabit/soundcloud_page_mapping.csv', index=False)

## Match soundcloud to page

## Clypit sounds

In [267]:
#taken from https://github.com/0x27/clypdown/blob/master/clypdown.py

def download(mp3_url, filename):
    try:
        r = requests.get(url=mp3_url, stream=True)
        with open(filename, 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    f.flush()
    except Exception as e:
        print ("{-} Something has gone horribly wrong! Please report on the github issue tracker with the following backtrace: \n%s" %(e))
    
def get_mp3_url(url):
    content_id = url.replace("https://clyp.it/", "")
    try:
        r = requests.get(url="https://api.clyp.it/%s" %(content_id))    
    except Exception as e:
        print ("{-} Something has gone horribly wrong! Please report on the github issue tracker with the following backtrace: \n%s" %(e))
    fucking_json = json.loads(r.text)
    try:
        song_title = fucking_json['Title']
        mp3_url = fucking_json['Mp3Url']
    except Exception as e:
        print ("{-} Something has gone horribly wrong! Please report on the github issue tracker with the following backtrace: \n%s" %(e))
        raise KeyError

    return song_title, mp3_url

def main(args):
    if len(args) != 2:
        sys.exit("%s https://clyp.it/lolwat" %(args[0]))
    song_title, mp3_url = get_mp3_url(url=args[1])
    download(mp3_url=mp3_url, title=song_title)


In [183]:
clypit_re = url_re = re.compile(r"(https?://(?:[\w\.]+\.)?clyp\.it/[\w]+\b)(?:[-a-zA-Z0-9()@:%_\+.~#?&/=]*)")
re.findall(clypit_re, 'https://clyp.it/wjrmj0au/widget')

['https://clyp.it/wjrmj0au']

In [372]:
clypit_sounds = []
for i, title, link in tqdm(list(wp_df.itertuples()), total=len(wp_df)):
    #print(i, title, link)
    wp_path = f'data/arabit/word_pages/{str(i+1)}.html'
    with open(wp_path, 'r') as f:
        wp_html = f.read()
    soup = BeautifulSoup(wp_html, 'html.parser')
    article = soup.find("article")
    sound_link = re.findall(clypit_re, article.prettify())
    clypit_sounds.append((i+1, title, link, sound_link))


100%|███████████████████████████████████████████████████████████████████████████████| 2302/2302 [01:52<00:00, 20.52it/s]


In [192]:
len([1 for s in clypit_sounds if s[3]!=[]])

1600

In [194]:
len([1 for s in clypit_sounds if len(s[3])>1])

320

In [195]:
clypit_sounds[:5]

[(1,
  'תודה',
  'https://arabit.me/%d7%aa%d7%95%d7%93%d7%94/',
  ['https://clyp.it/3lxfc3rh']),
 (2,
  'מערה',
  'https://arabit.me/%d7%9e%d7%a2%d7%a8%d7%94/',
  ['https://clyp.it/qgqnj2zc']),
 (3,
  'כיסא נדנדה',
  'https://arabit.me/%d7%9b%d7%99%d7%a1%d7%90-%d7%a0%d7%93%d7%a0%d7%93%d7%94/',
  []),
 (4,
  'הצטרף',
  'https://arabit.me/%d7%94%d7%a6%d7%98%d7%a8%d7%a3/',
  ['https://clyp.it/yq0whjd5']),
 (5,
  'טלפון',
  'https://arabit.me/%d7%98%d7%9c%d7%a4%d7%95%d7%9f/',
  ['https://clyp.it/qfzesdvu'])]

In [158]:
len(os.listdir('data/arabit/word_pages/'))

2303

In [212]:
len([1 for s, c in zip(sounds, clypit_sounds) if s[3]==[] and c[3]==[]])

14

In [213]:
[(c[1], c[2]) for s, c in zip(sounds, clypit_sounds) if s[3]==[] and c[3]==[]]

[('הרה-אסון',
  'https://arabit.me/%d7%94%d7%a8%d7%94-%d7%90%d7%a1%d7%95%d7%9f/'),
 ('זכינו!', 'https://arabit.me/%d7%96%d7%9b%d7%99%d7%a0%d7%95/'),
 ('התעורר!', 'https://arabit.me/%d7%94%d7%aa%d7%a2%d7%95%d7%a8%d7%a8-2/'),
 ('דירת יוקרה',
  'https://arabit.me/%d7%93%d7%99%d7%a8%d7%aa-%d7%99%d7%95%d7%a7%d7%a8%d7%94/'),
 ('היריון', 'https://arabit.me/%d7%94%d7%99%d7%a8%d7%99%d7%95%d7%9f-2/'),
 ('פאודה', 'https://arabit.me/%d7%a4%d7%90%d7%95%d7%93%d7%94/'),
 ('לקוח', 'https://arabit.me/%d7%9c%d7%a7%d7%95%d7%97/'),
 ('לאן?', 'https://arabit.me/%d7%9c%d7%90%d7%9f/'),
 ('קולפן קישואים',
  'https://arabit.me/%d7%a7%d7%95%d7%9c%d7%a4%d7%9f-%d7%a7%d7%99%d7%a9%d7%95%d7%90%d7%99%d7%9d/'),
 ('ילדה קטנה',
  'https://arabit.me/%d7%99%d7%9c%d7%93%d7%94-%d7%a7%d7%98%d7%a0%d7%94/'),
 ('ביטויים ומטבעות לשון',
  'https://arabit.me/%d7%91%d7%99%d7%98%d7%95%d7%99%d7%99%d7%9d-%d7%95%d7%9e%d7%98%d7%91%d7%a2%d7%95%d7%aa-%d7%9c%d7%a9%d7%95%d7%9f/'),
 ('דרך', 'https://arabit.me/%d7%93%d7%a8%d7%9a/'),
 ('תודה ל

In [258]:
clypit_urls_for_download = []
for c in clypit_sounds:
    links = c[3]
    for link in links:
        save_name = link.split('.it/')[1]
        clypit_urls_for_download.append((link, save_name))
        
clypit_urls_for_download = list(set(clypit_urls_for_download))

In [ ]:
clypit_metadata = []

In [273]:
from concurrent.futures import ThreadPoolExecutor

def save_clypit(params):
    link, save_name = params
    try:
        song_title, mp3_url = get_mp3_url(url=link)
    except:
        return
    output_name = song_title.replace('/', '-')+'.mp3'
    output_path = f'data/arabit/clypit/{output_name}'
    if not os.path.exists(output_path): 
        download(mp3_url, output_path)
        with open(output_path, 'w') as f:
            f.write(word_page_html.text)
    md_name = output_name.replace('.mp3', '.json')
    metadata_output_path = f'data/arabit/clypit_metadata/{md_name}'
    with open(metadata_output_path, 'w') as f:
        json.dump({'link': link, 'song_title': song_title, 'mp3_url': mp3_url, 'output_name': output_name}, f, indent=4)

with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(save_clypit, clypit_urls_for_download), total=len(clypit_urls_for_download)))

  7%|█████▋                                                                           | 98/1404 [00:03<00:31, 41.40it/s]

{-} Something has gone horribly wrong! Please report on the github issue tracker with the following backtrace: 
'Title'
{-} Something has gone horribly wrong! Please report on the github issue tracker with the following backtrace: 
'Title'


 13%|██████████▎                                                                     | 181/1404 [00:05<00:24, 50.36it/s]

{-} Something has gone horribly wrong! Please report on the github issue tracker with the following backtrace: 
'Title'


 18%|██████████████▎                                                                 | 251/1404 [00:06<00:24, 47.98it/s]

...

 93%|█████████████████████████████████████████████████████████████████████████▎     | 1302/1404 [00:28<00:02, 47.01it/s]

{-} Something has gone horribly wrong! Please report on the github issue tracker with the following backtrace: 
'Title'


 94%|██████████████████████████████████████████████████████████████████████████▍    | 1322/1404 [00:29<00:01, 46.27it/s]

{-} Something has gone horribly wrong! Please report on the github issue tracker with the following backtrace: 
'Title'


100%|███████████████████████████████████████████████████████████████████████████████| 1404/1404 [00:31<00:00, 45.24it/s]


In [367]:
clypit_sounds_metadata = []
for file in os.listdir('data/arabit/clypit_metadata'):
    if not '.json' in file:
        continue
    with open('data/arabit/clypit_metadata/'+file, 'r') as f:
        md = json.load(f)
    md['local_clypit_file'] = ('data/arabit/clypit_metadata/'+file).replace('.json', '.mp3').replace('_metadata/', '/')
    clypit_sounds_metadata.append(md)
    
clypit_df = pd.DataFrame(clypit_sounds_metadata)
clypit_df

,link,song_title,mp3_url,output_name,local_clypit_file
0,https://clyp.it/kld0eorr,ברג',https://a.clyp.it/kld0eorr.mp3,ברג'.mp3,data/arabit/clypit/ברג'.mp3
1,https://clyp.it/ilsyy0aa,בלאש,https://audio.clyp.it/ilsyy0aa.mp3?Expires=170...,בלאש.mp3,data/arabit/clypit/בלאש.mp3
2,https://clyp.it/qmtjcccr,ד׳וק,https://audio.clyp.it/qmtjcccr.mp3?Expires=170...,ד׳וק.mp3,data/arabit/clypit/ד׳וק.mp3
3,https://clyp.it/tiq5eq4j,פטאיר,https://a.clyp.it/tiq5eq4j.mp3,פטאיר.mp3,data/arabit/clypit/פטאיר.mp3
4,https://clyp.it/f2jolwl2,ג׳והר,https://audio.clyp.it/f2jolwl2.mp3?Expires=170...,ג׳והר.mp3,data/arabit/clypit/ג׳והר.mp3
...,...,...,...,...,...
1374,https://clyp.it/l5owm2jn,סמיתו אלכלב בובי,https://audio.clyp.it/l5owm2jn.mp3?Expires=170...,סמיתו אלכלב בובי.mp3,data/arabit/clypit/סמיתו אלכלב בובי.mp3
1375,https://clyp.it/ol0silvr,ח׳ליפה,https://audio.clyp.it/ol0silvr.mp3?Expires=170...,ח׳ליפה.mp3,data/arabit/clypit/ח׳ליפה.mp3
1376,https://clyp.it/uem0rrak,חט'ירה,https://audio.clyp.it/uem0rrak.mp3?Expires=170...,חט'ירה.mp3,data/arabit/clypit/חט'ירה.mp3
1377,https://clyp.it/x52i1etn,נקאבה,https://audio.clyp.it/x52i1etn.mp3?Expires=170...,נקאבה.mp3,data/arabit/clypit/נקאבה.mp3


In [380]:
clypit_link_to_title = clypit_df.set_index('link').song_title.to_dict()
song_title_to_clypit_file = clypit_df.set_index('song_title').local_clypit_file.to_dict()
clypit_link_to_title

{'https://clyp.it/kld0eorr': "ברג'",
 'https://clyp.it/ilsyy0aa': 'בלאש',
 'https://clyp.it/qmtjcccr': 'ד׳וק',
 'https://clyp.it/tiq5eq4j': 'פטאיר',
 'https://clyp.it/f2jolwl2': 'ג׳והר',
 'https://clyp.it/riv2uht4': 'אשקר',
 'https://clyp.it/xny1xhoq': 'זאח',
 'https://clyp.it/iav10n3x': 'תוזיע',
 'https://clyp.it/hxed5xcf': 'תסריח',
 'https://clyp.it/z3kcszc1': 'פרקה',
 'https://clyp.it/qbuuchao': 'ח׳רפ',
 'https://clyp.it/wr35ms0w': 'כאמל אלאוצאף',
 'https://clyp.it/sbruca4p': 'חדאד',
 'https://clyp.it/dtvty453': 'אסד',
 'https://clyp.it/g1wadnbi': 'דולה',
 'https://clyp.it/hdzssbvb': 'חראם',
 'https://clyp.it/t1hofaza': 'מכפוף',
 'https://clyp.it/3tizrxcq': 'ג׳אג׳',
 'https://clyp.it/sgaj5vlb': 'תפרג׳',
 'https://clyp.it/zqrdhsay': 'חואג׳ה',
 'https://clyp.it/erifsjcv': 'וחי',
 'https://clyp.it/4kqrhrv3': 'דרבכה',
 'https://clyp.it/10dd3llw': "פִנְגַ'אן",
 'https://clyp.it/huengh5c': 'חאפיה',
 'https://clyp.it/0itzhqgm': 'לע׳ז',
 'https://clyp.it/2cqttciy': 'פג׳עאן',
 'https://clyp.

In [381]:
page_to_clypit_titles = []
for page_num, page_title, page_url, links in clypit_sounds:
    #titles = clypit_df[clypit_df.link.isin(links)].song_title.tolist()
    titles = [clypit_link_to_title[link] for link in links if link in clypit_link_to_title]
    if len(titles)==0:
        continue
    elif len(titles)==1:
        chosen_title = titles[0]
    else:
        chosen_title = titles[0]
        print(page_title, titles, '-->', chosen_title)
    page_to_clypit_titles.append( (page_num, page_title, chosen_title, song_title_to_clypit_file[chosen_title]))    

תודה ['שכרן', 'שכרן'] --> שכרן
הצטרף ['צ׳ם', 'צ׳ם'] --> צ׳ם
פטפטן; נודניק ['זנאן', 'זנאן'] --> זנאן
לחץ ["צ'ע'ט אלדם", "צ'ע'ט אלדם"] --> צ'ע'ט אלדם
טעים ['זאכי', 'זאכי'] --> זאכי
שמן ['זיתון', 'זית'] --> זיתון
מטופש ['באיח׳', 'באיח׳'] --> באיח׳
ענו ['ודאע', 'ודיע'] --> ודאע
בלתי מזוהה ['ג׳אהל', "מג'הול", "ינתהי אלעלם חית' יבדא אלג'הל"] --> ג׳אהל
בחורות ['צבי', 'צלי ע אלנבי'] --> צבי
ברכוש ולא בנפש ['מואל', 'ראסמאל'] --> מואל
חייזר ["כלאם פאצ'י", 'מכוכ אלפצ׳אא׳', "כלאם פאצ'י"] --> כלאם פאצ'י
חומרי ['מד', "תע'דא ותמדא תעשא ותמשא"] --> מד
בחורים שופעי תכונות גבריות ['נשמי', 'חוראני'] --> נשמי
זנב ["ד'יל אלחצאן", 'אִללִּי עַנְדֹה כְּתִיר חִנֶּה בִּחַנִּי דֵיל חְמַארֹה'] --> ד'יל אלחצאן
מדרגות ['דרג׳את אלחרארה', 'דראג׳ה נאריה'] --> דרג׳את אלחרארה
טיפ ['כרם', 'כרים'] --> כרם
עור ['בשאר', 'בשר'] --> בשאר
אינקובטור ['חצ׳ן', "אלקנפד'"] --> חצ׳ן
טעים, בריא ['הנא', 'ניאלכ'] --> הנא
שוק (איבר) ['סואקה', 'סואקה'] --> סואקה
מרגיש ['חסאסיה', 'חסאסיה'] --> חסאסיה
סמטה ['זקאק', 'זקה'] --> זקאק
מרוצה ['

In [382]:
clypit_page_mapping = pd.DataFrame(page_to_clypit_titles, columns=["page_num", "page_title", "sound_title", "local_file"])
clypit_page_mapping.head()

,page_num,page_title,sound_title,local_file
0,1,תודה,שכרן,data/arabit/clypit/שכרן.mp3
1,2,מערה,ע׳ור מן ושי,data/arabit/clypit/ע׳ור מן ושי.mp3
2,4,הצטרף,צ׳ם,data/arabit/clypit/צ׳ם.mp3
3,5,טלפון,האתפ ח׳לוי,data/arabit/clypit/האתפ ח׳לוי.mp3
4,6,פטפטן; נודניק,זנאן,data/arabit/clypit/זנאן.mp3


In [383]:
clypit_page_mapping.to_csv('data/arabit/clypit_page_mapping.csv', index=False)

## TODO: Choose sound file by the first between all of them!!!


In [436]:
soundcloud_re.pattern

'https?://(?:[\\w\\.]+\\.)?soundcloud\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&/=]*)'

In [437]:
clypit_re.pattern

'(https?://(?:[\\w\\.]+\\.)?clyp\\.it/[\\w]+\\b)(?:[-a-zA-Z0-9()@:%_\\+.~#?&/=]*)'

In [441]:
sc_clyp_re = re.compile(fr'(?:{soundcloud_re.pattern})|(?:{clypit_re.pattern[1:-1]})|title=.* by Arabit')

In [443]:
all_sound_links = []
for i, title, link in tqdm(list(wp_df.itertuples()), total=len(wp_df)):
    #print(i, title, link)
    wp_path = f'data/arabit/word_pages/{str(i+1)}.html'
    with open(wp_path, 'r') as f:
        wp_html = f.read()
    soup = BeautifulSoup(wp_html, 'html.parser')
    article = soup.find("article")
    sound_link = re.findall(sc_clyp_re, article.prettify())
    sound = article.find("iframe", title=re.compile(".* by Arabit"))
    if sound is not None:
        sound = sound.get("title")
    else:
        sound = None
    all_sound_links.append((i, title, sound, sound_link))

  0%|                                                                                  | 3/2302 [00:00<01:40, 22.95it/s]

['https://on.soundcloud.com/s1Lxr?fbclid=IwAR2rKTdcdVf0Q2KqfKO5sd8ryS9v8YkyM4vTCNf1mM5GDWK7PFamfmh4-tI', 'https://on.soundcloud.com/s1Lxr', 'https://clyp.it/3lxfc3rh?fbclid=IwAR3TJCmLJYvdYWucL6DvETInvQ1WNwPJw6Afsw3sLjOwZdlTzB97tmec_Gs', 'https://clyp.it/3lxfc3rh']
['https://on.soundcloud.com/Jag6v?fbclid=IwAR2DYrZ5SIN2FY_glT5_Dk9PgcXtkwbg0fjPc6ii808n_QMext7GPH73qjI', 'https://on.soundcloud.com/Jag6v', 'https://clyp.it/qgqnj2zc']
['https://on.soundcloud.com/9RqRd?fbclid=IwAR1tQ0S383bBOnmGrl1Ra12LwOxaTUeb79i-3j_nKnPlAvk2vUR77ORw3ko', 'https://on.soundcloud.com/9RqRd']
['https://on.soundcloud.com/HAA6e?fbclid=IwAR1e3r3Ng81_W_iHjGE_2OSld40V3zYkslbvG9ODc29ZiLIcPVAD2wRVQCk', 'https://on.soundcloud.com/HAA6e', 'https://clyp.it/yq0whjd5?fbclid=IwAR2KNG7EOrkm_XuA3-mh1xKRGxCZD7xSMA4aeO-Z79QOvJ7IyMAtgTu-s8g', 'https://clyp.it/yq0whjd5']
['https://on.soundcloud.com/SDpcs', 'https://clyp.it/qfzesdvu/widget']


  0%|▎                                                                                 | 9/2302 [00:00<01:34, 24.39it/s]

['https://on.soundcloud.com/9LNpN?fbclid=IwAR2zSqZ9LWTFOfFOH-XfZQvdHSqdWsF7MKfoo0M2O40SKioF71QEr341k7k', 'https://on.soundcloud.com/9LNpN', 'https://clyp.it/uh1mau4v?fbclid=IwAR2YoHDSyt5ag8UJ7viB-NSQMAX3_kribJww5OaarWqRBdIw7sP-r_tf5Lw', 'https://clyp.it/uh1mau4v']
['https://on.soundcloud.com/2vKQi?fbclid=IwAR3dck487XOcFSWEtf-NQN48qJFiOXA_yns3kmW7aCepp8PSxfPLvmOuKf0', 'https://on.soundcloud.com/2vKQi', 'https://clyp.it/juwp1g30']
['https://on.soundcloud.com/Duit7']
['https://on.soundcloud.com/cEU6s?fbclid=IwAR0Gpo1KeoWhxdLpxiiYyfqUjGlgo56wDFajMAoiIdPv96r4CBu2GBH4b08', 'https://on.soundcloud.com/cEU6s', 'https://clyp.it/stn112my']
['https://on.soundcloud.com/nCZCk']
['https://on.soundcloud.com/FbWTy', 'https://clyp.it/ojzvsgqk/widget']


  1%|▌                                                                                | 15/2302 [00:00<01:29, 25.47it/s]

...

['https://w.soundcloud.com/player/?visual=false&amp', 'title="אסתע\'את\'ה by Arabit']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="לקאח by Arabit']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="סאא\'ל by Arabit']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="ג\'אאזה by Arabit']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="חבשה by Arabit']
['https://clyp.it/uzu2gsbd/widget']


100%|██████████████████████████████████████████████████████████████████████████████▊| 2295/2302 [01:53<00:00, 23.30it/s]

['https://w.soundcloud.com/player/?visual=false&amp', 'title="תרשיח by Arabit', 'https://clyp.it/amzf5wrk/widget']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="אחסן by Arabit', 'https://clyp.it/vegkxc4n/widget']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="לג\'נה by Arabit']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="אונטה by Arabit']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="מתח\'וזק by Arabit', 'https://clyp.it/etd1cb0y/widget']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="כרמאל עיונכ by Arabit', 'https://clyp.it/3ifalzki/widget', 'https://clyp.it/rmgl4yvl/widget']


100%|███████████████████████████████████████████████████████████████████████████████| 2302/2302 [01:53<00:00, 20.20it/s]

['https://clyp.it/hptwh3no/widget', 'https://clyp.it/r2wvv5tw/widget']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="מחבוס by Arabit']
['https://clyp.it/rhwou5up/widget']
['https://clyp.it/tfoxobbg/widget']
['https://w.soundcloud.com/player/?visual=false&amp', 'title="נבי by Arabit', 'https://clyp.it/bx2tcfs4/widget', 'https://clyp.it/xhitbh2m/widget']


## Collect all to one csv

In [444]:
wdf = pd.DataFrame(words, columns=["page_num", "title", "link", "image", "caption", "caption_part", "arabic_word"])
wdf.head()

,page_num,title,link,image,caption,caption_part,arabic_word
0,0,תודה,https://arabit.me/%d7%aa%d7%95%d7%93%d7%94/,https://arabit.me/wp-content/uploads/2024/01/ת...,תודה,תודה,شُكْرًا
1,1,מערה,https://arabit.me/%d7%9e%d7%a2%d7%a8%d7%94/,https://arabit.me/wp-content/uploads/2024/01/מ...,מערה,מערה,مغارة
2,2,כיסא נדנדה,https://arabit.me/%d7%9b%d7%99%d7%a1%d7%90-%d7...,https://arabit.me/wp-content/uploads/2024/01/כ...,כיסא נדנדה,כיסא נדנדה,كُرْسِيّ رَجَّاج
3,3,הצטרף,https://arabit.me/%d7%94%d7%a6%d7%98%d7%a8%d7%a3/,https://arabit.me/wp-content/uploads/2024/01/ה...,הצטרף,הצטרף,اِنْضَمّ
4,4,טלפון,https://arabit.me/%d7%98%d7%9c%d7%a4%d7%95%d7%9f/,https://arabit.me/wp-content/uploads/2024/01/ט...,טלפון,טלפון,هاتف


In [445]:
wdf.link.nunique()

2286

In [414]:
# add local image path

In [446]:
wdf["local_image_path"] = wdf.image.apply(lambda x: f'data/arabit/images/{x.split("/")[-1]}')
wdf["local_image_path_small"] = wdf.image.apply(lambda x: f'data/arabit/images_smaller/{x.split("/")[-1]}')
wdf["local_image_path_xs"] = wdf.image.apply(lambda x: f'data/arabit/images_xs/{x.split("/")[-1]}')
wdf.head()

,page_num,title,link,image,caption,caption_part,arabic_word,local_image_path,local_image_path_small,local_image_path_xs
0,0,תודה,https://arabit.me/%d7%aa%d7%95%d7%93%d7%94/,https://arabit.me/wp-content/uploads/2024/01/ת...,תודה,תודה,شُكْرًا,data/arabit/images/תודה.jpg,data/arabit/images_smaller/תודה.jpg,data/arabit/images_xs/תודה.jpg
1,1,מערה,https://arabit.me/%d7%9e%d7%a2%d7%a8%d7%94/,https://arabit.me/wp-content/uploads/2024/01/מ...,מערה,מערה,مغارة,data/arabit/images/מערה.jpg,data/arabit/images_smaller/מערה.jpg,data/arabit/images_xs/מערה.jpg
2,2,כיסא נדנדה,https://arabit.me/%d7%9b%d7%99%d7%a1%d7%90-%d7...,https://arabit.me/wp-content/uploads/2024/01/כ...,כיסא נדנדה,כיסא נדנדה,كُرْسِيّ رَجَّاج,data/arabit/images/כיסא-נדנדה.jpg,data/arabit/images_smaller/כיסא-נדנדה.jpg,data/arabit/images_xs/כיסא-נדנדה.jpg
3,3,הצטרף,https://arabit.me/%d7%94%d7%a6%d7%98%d7%a8%d7%a3/,https://arabit.me/wp-content/uploads/2024/01/ה...,הצטרף,הצטרף,اِنْضَمّ,data/arabit/images/הצטרף.jpg,data/arabit/images_smaller/הצטרף.jpg,data/arabit/images_xs/הצטרף.jpg
4,4,טלפון,https://arabit.me/%d7%98%d7%9c%d7%a4%d7%95%d7%9f/,https://arabit.me/wp-content/uploads/2024/01/ט...,טלפון,טלפון,هاتف,data/arabit/images/טלפון.jpg,data/arabit/images_smaller/טלפון.jpg,data/arabit/images_xs/טלפון.jpg


In [462]:
all_sound_mapping = {}
clypit_link_to_local_file = clypit_df.set_index('link').local_clypit_file.to_dict()
soundcloud_link_to_local_file = sc_titles_df.set_index('original_link').local_path.to_dict()
soundcloud_clean_link_to_local_file = sc_titles_df.set_index('clean_link').local_path.to_dict()
soundcloud_url_to_local_file = sc_titles_df.set_index('url').local_path.to_dict()
all_sound_mapping = clypit_link_to_local_file | soundcloud_link_to_local_file | soundcloud_clean_link_to_local_file | soundcloud_url_to_local_file
len(all_sound_mapping)

2554

In [467]:
all_page_sound_mapping = {}
for page_num, title, sound, sound_links in all_sound_links:
    sound_links = [s for s in sound_links if '/player/' not in s]
    first_sound = sound_links[0] if sound_links!=[] else sound
    if first_sound is None:
        continue
    try:
        if first_sound.startswith('http'):
            local_path = all_sound_mapping.get(first_sound)
            if local_path is None:
                clean_link = re.sub('\?.*$', '', first_sound)
                clean_link = re.sub('/widget$', '', first_sound)
                local_path = all_sound_mapping[clean_link]
        else:
            clean_title = first_sound.replace('title="', '').replace(' by Arabit', '').strip()
            local_path = 'data/arabit/soundcloud/' + clean_title.replace("/", "") + '.mp3'
        all_page_sound_mapping[page_num-1] = local_path
    except Exception as e:
        print(e)
        #print(page_num, title, sound, sound_links, first_sound)
    
wdf['local_sound_path'] = wdf.page_num.map(all_page_sound_mapping)
wdf.head()
        

'https://on.soundcloud.com/rfJ7z?fbclid=IwAR1FcKuy5z-hfmKGSrMwTguCp0RwhsPUPym6iOPV-B6cEIAaq9u5DVrptYc'
'https://on.soundcloud.com/EUpc7?fbclid=IwAR19EfPT04xkk8-aFHYFU96aBUnbBtNNwt-a6IeR73ogVL09dCiJaz8Cx10'
'https://on.soundcloud.com/z2VqL?fbclid=IwAR1Bp4pkTINmctuAwyzaU4cPRFZps8SWRZQBa8iIGE2wmjIVECDZpl8paEk'
'https://clyp.it/5opecyf5'
'https://on.soundcloud.com/bw4UB?fbclid=IwAR0v0NvBpkoST6aPCOEWUgagYfXThpilUiRoZzMyuEmk81NUicNeIvOt7zs'
'https://on.soundcloud.com/7BQ91?fbclid=IwAR3RXXpSPlfnvN-p_bPEktDYprpa9GnXG5rea-xMqYXKeXeM9IywWxlgL6s'
'https://clyp.it/2qesgnsz'
'https://clyp.it/c51srbrg'
'https://clyp.it/sjfig2fj'
'https://clyp.it/c5pb5vjr'
'https://clyp.it/vzocynfp'
'https://clyp.it/o545hhtu'
'https://clyp.it/uqbtz2yh'
'https://clyp.it/1qg0rsea'


,page_num,title,link,image,caption,caption_part,arabic_word,local_image_path,local_image_path_small,local_image_path_xs,local_sound_path
0,0,תודה,https://arabit.me/%d7%aa%d7%95%d7%93%d7%94/,https://arabit.me/wp-content/uploads/2024/01/ת...,תודה,תודה,شُكْرًا,data/arabit/images/תודה.jpg,data/arabit/images_smaller/תודה.jpg,data/arabit/images_xs/תודה.jpg,data/arabit/soundcloud/שכרן (תודה).mp3
1,1,מערה,https://arabit.me/%d7%9e%d7%a2%d7%a8%d7%94/,https://arabit.me/wp-content/uploads/2024/01/מ...,מערה,מערה,مغارة,data/arabit/images/מערה.jpg,data/arabit/images_smaller/מערה.jpg,data/arabit/images_xs/מערה.jpg,data/arabit/soundcloud/מע׳ארה (מערה).mp3
2,2,כיסא נדנדה,https://arabit.me/%d7%9b%d7%99%d7%a1%d7%90-%d7...,https://arabit.me/wp-content/uploads/2024/01/כ...,כיסא נדנדה,כיסא נדנדה,كُرْسِيّ رَجَّاج,data/arabit/images/כיסא-נדנדה.jpg,data/arabit/images_smaller/כיסא-נדנדה.jpg,data/arabit/images_xs/כיסא-נדנדה.jpg,data/arabit/soundcloud/כרסי רג׳אג׳ (כסא נדנדה)...
3,3,הצטרף,https://arabit.me/%d7%94%d7%a6%d7%98%d7%a8%d7%a3/,https://arabit.me/wp-content/uploads/2024/01/ה...,הצטרף,הצטרף,اِنْضَمّ,data/arabit/images/הצטרף.jpg,data/arabit/images_smaller/הצטרף.jpg,data/arabit/images_xs/הצטרף.jpg,data/arabit/soundcloud/אנצ׳ם (הצטרף).mp3
4,4,טלפון,https://arabit.me/%d7%98%d7%9c%d7%a4%d7%95%d7%9f/,https://arabit.me/wp-content/uploads/2024/01/ט...,טלפון,טלפון,هاتف,data/arabit/images/טלפון.jpg,data/arabit/images_smaller/טלפון.jpg,data/arabit/images_xs/טלפון.jpg,data/arabit/soundcloud/האתף (טלפון).mp3


In [469]:
wdf.tail()


,page_num,title,link,image,caption,caption_part,arabic_word,local_image_path,local_image_path_small,local_image_path_xs,local_sound_path
2281,2297,איכס,https://arabit.me/%d7%90%d7%99%d7%9b%d7%a1/,https://arabit.me/wp-content/uploads/2021/01/1...,איכס,איכס,اخسا,data/arabit/images/139431422_952611835272677_2...,data/arabit/images_smaller/139431422_952611835...,data/arabit/images_xs/139431422_95261183527267...,data/arabit/clypit/אח׳ס.mp3
2282,2298,כלוא,https://arabit.me/%d7%9b%d7%9c%d7%95%d7%90/,https://arabit.me/wp-content/uploads/2021/01/1...,כלוא,כלוא,مَحْبُوس,data/arabit/images/138843949_952605831939944_5...,data/arabit/images_smaller/138843949_952605831...,data/arabit/images_xs/138843949_95260583193994...,data/arabit/soundcloud/מחבוס.mp3
2283,2299,נעל ישנה,https://arabit.me/%d7%a0%d7%a2%d7%9c-%d7%99%d7...,https://arabit.me/wp-content/uploads/2021/01/1...,נעל ישנה,נעל ישנה,صُرْمَاية,data/arabit/images/139030053_952606478606546_3...,data/arabit/images_smaller/139030053_952606478...,data/arabit/images_xs/139030053_95260647860654...,data/arabit/clypit/צרמאיה.mp3
2284,2300,טבח,https://arabit.me/%d7%98%d7%91%d7%97/,https://arabit.me/wp-content/uploads/2021/01/1...,טבח,טבח,مَذْبَحَة,data/arabit/images/118540761_853060481894480_5...,data/arabit/images_smaller/118540761_853060481...,data/arabit/images_xs/118540761_85306048189448...,data/arabit/clypit/מד'בחה.mp3
2285,2301,נביא,https://arabit.me/%d7%a0%d7%91%d7%99%d7%90/,https://arabit.me/wp-content/uploads/2021/01/1...,נביא,נביא,نَبِيّ,data/arabit/images/136962660_949236392276888_3...,data/arabit/images_smaller/136962660_949236392...,data/arabit/images_xs/136962660_94923639227688...,data/arabit/soundcloud/נבי.mp3


In [468]:
wdf.to_csv("data/arabit/words.csv", index=False)

## Create Anki Deck

In [605]:
import random
import genanki
random.seed(64)

# model_id = random.randrange(1 << 30, 1 << 31)
# deck_id = random.randrange(1 << 30, 1 << 31)
model_id = 1359479921
deck_id = 1491813214

print(model_id)
print(deck_id)


my_model = genanki.Model(
  model_id,
  'Arabit',
  fields=[
    {'name': 'Recording'},
    {'name': 'Image'},
    {'name': 'URL'},
    {'name': 'Arabic', 'rtl': True, 'html': True},
    {'name': 'Hebrew', 'rtl': True, 'html': True},
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Recording}}',
      'afmt': '{{FrontSide}}'
              '<br><hr id="answer"><br>'
              '<div style="text-align:center;font-size:30px">'
              '{{Image}}'
              '<br>'
              '<div style="color:#00055f"><span dir="rtl">&rlm;{{Arabic}}&rlm;</span></div>'
              '<span dir="rtl">&rlm;{{Hebrew}}&rlm;</span>'
              '<br>'
              '<a href="{{URL}}">arabit.me</a>'
              '</div>',
    },
  ],
    css='''.card {
 font-family: abraham;
 font-size: 30px;
 text-align: center
 
 }
 
 @font-face {
    font-family: abraham;
    src: url('_Abraham-Light.otf')
}
''')

1359479921
1491813214


In [606]:
my_deck = genanki.Deck(
  deck_id,
  'Arabit')


In [607]:
wdf.dropna().shape

(2150, 11)

In [608]:
wdf.shape

(2286, 11)

In [609]:
wdf.columns

Index(['page_num', 'title', 'link', 'image', 'caption', 'caption_part',
       'arabic_word', 'local_image_path', 'local_image_path_small',
       'local_image_path_xs', 'local_sound_path'],
      dtype='object')

In [610]:
columns_to_use = ['link', 'caption', 'arabic_word', 'local_image_path_small', 'local_sound_path']

In [611]:
media_files = []
for _, link, hebrew, arabic, image, sound in wdf[columns_to_use].dropna().itertuples():
    media_files.append(image)
    media_files.append(sound)
    my_note = genanki.Note(
        model=my_model,
        fields=[f'[sound:{sound.split("/")[-1]}]',
                f'<img src="{image.split("/")[-1]}">',
                link,
                html.escape(arabic),
                html.escape(hebrew)])
    my_deck.add_note(my_note)

In [612]:
my_package = genanki.Package(my_deck)
# my_package.media_files = (['data/arabit/clypit/' + file for file in os.listdir('data/arabit/clypit/')]
#                           + ['data/arabit/soundcloud/' + file for file in os.listdir('data/arabit/soundcloud/')]
#                           + ['data/arabit/images_smaller/' + file for file in os.listdir('data/arabit/images_smaller/')]
#                          )
my_package.media_files = media_files

In [613]:
my_package.write_to_file('data/arabit_recordings.apkg')